In [1]:
data_dir = "/harddisk/user/keminglu/evaluation_corpus"

In [3]:
import json
import os

In [4]:
file_list = os.listdir(os.path.join(data_dir, "wiki"))

In [15]:
data = []
for file in file_list:
    with open(os.path.join(data_dir, "wiki", file)) as f:
        data += [json.loads(line) for line in f.readlines()]

In [16]:
filtered_data = [each for each in data if each['n_ents'] > 0]

In [17]:
annotates = []
texts = []
for each in filtered_data:
    texts.append(" ".join(each["cleaned_text"]))
    del each["cleaned_text"]
    annotates.append(each)

In [18]:
with open("wiki_eval.txt", "w") as f:
    for text in texts:
        f.write(text+"\n")
with open("wiki_eval.ann", "w") as f:
    for ann in annotates:
        f.write(json.dumps(ann)+"\n")

In [29]:
from datasets import load_dataset
import os

In [31]:
dataset = load_dataset("conll2003", cache_dir=data_dir)

Found cached dataset conll2003 (/harddisk/user/keminglu/evaluation_corpus/conll2003/conll2003/1.0.0/9a4d16a94f8674ba3466315300359b0acd891b68b6c8743ddf60b9c702adce98)


  0%|          | 0/3 [00:00<?, ?it/s]

In [32]:
texts = [" ".join(tokens).lower() for tokens in dataset['test']['tokens']]

In [34]:
with open(os.path.join(data_dir, "conll03.txt"), "w") as f:
    for text in texts:
        f.write(text + "\n")

In [20]:
dataset = load_dataset("cnn_dailymail", "3.0.0", cache_dir=data_dir)

Found cached dataset cnn_dailymail (/harddisk/user/keminglu/evaluation_corpus/cnn_dailymail/3.0.0/3.0.0/1b3c71476f6d152c31c1730e83ccb08bcf23e348233f4fcc11e182248e6bf7de)


  0%|          | 0/3 [00:00<?, ?it/s]

In [28]:
texts = [line.replace("\n", "") for line in dataset['test']['highlights']]

In [30]:
with open(os.path.join(data_dir, "cnn_dailymail_highlights.txt"), "w") as f:
    for text in texts:
        f.write(text + "\n")

### Insert evaluate report to DB

In [22]:
from pymongo import MongoClient

In [23]:
mongodb_config = {"host": '10.12.192.31', "port": 27017}
client = MongoClient(**mongodb_config)
evaluation_collection = client['structure_lm']['evaluation']

In [30]:
with open("/harddisk/user/keminglu/evaluation_corpus/cnn_dailymail_highlights_output_with_evaluation.txt") as f:
    data = [json.loads(line) for line in f.readlines()]
for i in range(len(data)):
    data[i]["output"][0] = json.loads(data[i]["output"][0])
    data[i].update({"source": "cnn_dailymail_highlights"})

In [31]:
evaluation_collection.insert_many(data, ordered=True)

In [32]:
with open("/harddisk/user/keminglu/evaluation_corpus/wiki_eval_output_with_evaluation.txt") as f:
    data = [json.loads(line) for line in f.readlines()]
for i in range(len(data)):
    data[i]["output"][0] = json.loads(data[i]["output"][0])
    data[i].update({"source": "wiki"})

In [33]:
evaluation_collection.insert_many(data, ordered=True)